In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Simple_RLHF')

from google.colab import drive

drive.mount('/content/drive')
!ls -al /content/drive/MyDrive/Simple_RLHF/model/actor
!ls -al /content/drive/MyDrive/Simple_RLHF/model/
!ls -al /content/drive/MyDrive/Simple_RLHF/model/critic
!ls -al /content/drive/MyDrive/Simple_RLHF/model/rlhf



import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

Mounted at /content/drive
total 5139760
-rw------- 1 root root        749 Mar  9 01:55 config.json
-rw------- 1 root root        132 Mar  9 01:55 generation_config.json
-rw------- 1 root root 4994509120 Mar  9 01:55 model-00001-of-00002.safetensors
-rw------- 1 root root  268568856 Mar  9 01:56 model-00002-of-00002.safetensors
-rw------- 1 root root      33832 Mar  9 01:56 model.safetensors.index.json
total 1293915
drwx------ 2 root root       4096 Mar  9 01:55 actor
-rw------- 1 root root 1324960543 Mar  9 13:16 critic
drwx------ 2 root root       4096 Mar  9 16:29 rlhf
-rw------- 1 root root 1324960543 Mar  9 13:16 /content/drive/MyDrive/Simple_RLHF/model/critic
total 5139760
-rw------- 1 root root        778 Mar 10 14:59 config.json
-rw------- 1 root root        132 Mar 10 14:59 generation_config.json
-rw------- 1 root root 4994509120 Mar 10 15:20 model-00001-of-00002.safetensors
-rw------- 1 root root  268568856 Mar 10 15:20 model-00002-of-00002.safetensors
-rw------- 1 root root  

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

(tensor([   1,    1,    0, 9178,   32,   47]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<pad><pad><s>how are you')

In [3]:
import torch
print(torch.cuda.is_available())  # 应该输出True
from transformers import AutoModelForCausalLM

model_actor = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Simple_RLHF/model/rlhf',
                                                   torch_dtype=torch.float16,
                                                   device_map='cuda')

True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import json

with open('/content/drive/MyDrive/Simple_RLHF/dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=8)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(8,
 {'prompt': 'Human: context= CREATE TABLE table_name_42 (floors VARCHAR, location VARCHAR) question= How many floors are in little Italy? Assistant:',
  'chosen': 'SELECT COUNT(floors) FROM table_name_42 WHERE location = "little italy"',
  'rejected': '',
  'response': 'SELECT COUNT(floors) FROM table_name_42 WHERE location = "little italy"'})

In [5]:
for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=256)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = model_actor.generate(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    max_length=512,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id)

    generate = generate[0, 256:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select FLOORS from TABLE_NAME_42 where LOCATION = "LITTLE ITALY"</s>
SELECT COUNT(floors) FROM table_name_42 WHERE location = "little italy"
select min(pick) from TABLE_NAME_13 where PLAYER = "Arturo McDowell" and POSITION = "OF" and TEAM = "SF Giants"</s>
SELECT MIN(pick) FROM table_name_13 WHERE position = "of" AND team = "san francisco giants" AND player = "arturo mcdowell"
select SET_3 from TABLE_NAME_54 where SET_2 = "NA" and SET_5 = "21-25" and SET_1 = "25-16"</s>
SELECT set_3 FROM table_name_54 WHERE set_5 = "na" AND set_1 = "21-25" AND set_2 = "25-16"
select STATUS from TABLE_NAME_8 where COUNTRY = "bel" and TRANSFER_WINDOW = "wintry"</s>
SELECT status FROM table_name_8 WHERE country = "bel" AND transfer_window = "winter"
select COMPETITION from TABLE_NAME_73 where GOAL = "GOAL 8"</s>
SELECT competition FROM table_name_73 WHERE goal = 8
select INCUMBENT from TABLE_1341549_10 where FIRST_ELECTED = "incumbent"</s>
SELECT incumbent FROM table_1341549_10 WHERE first_elected = "1954